In [0]:
application_id="257f1bbb-d8da-46ed-9dc1-a417530e9324"
directory_id="654ad10e-3170-4844-bb2d-b2c7a9ae5a8d"
service_credential = dbutils.secrets.get('blob-scope-rawstorage123','service-credential-project')

spark.conf.set("fs.azure.account.auth.type.rawstorage123.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.rawstorage123.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.rawstorage123.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.rawstorage123.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.rawstorage123.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://processed@rawstorage123.dfs.core.windows.net/processed_renamed"))

path,name,size,modificationTime
abfss://processed@rawstorage123.dfs.core.windows.net/processed_renamed/processed_sales_data.csv,processed_sales_data.csv,8381,1733292314000


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

sales_schema = StructType([
    StructField("TransactionID",StringType(), True),
    StructField("CustomerName",StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity",DoubleType(), True),
    StructField("Region", StringType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("TransactionDate", DateType(), True),
    StructField("TotalAmount", IntegerType(), True),
])

df = spark.read.csv(
    "abfss://processed@rawstorage123.dfs.core.windows.net/processed_renamed/processed_sales_data.csv",
    header=True,
    schema=sales_schema
)
df.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- TransactionDate: date (nullable = true)
 |-- TotalAmount: integer (nullable = true)



In [0]:
from pyspark.sql.functions import expr, year,col

# Clean Price Per Unit and derive Total Amount
#df = df.withColumn("TotalAmount", col("Quantity") * col("UnitPrice"))
df = df.withColumn("DiscountedAmount", col("TotalAmount") * 0.9)

df = df.filter(col("Quantity") > 0)

# Add Transaction Year
#df = df.withColumn("TransactionYear", year(col("TransactionDate")))

# Write to Delta Lake
#df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://staging@rawstorage123.dfs.core.windows.net/processed_sales_data.csv")

#output_path = "abfss://staging@rawstorage123.dfs.core.windows.net/processed_sales_data.csv"  # Replace with your desired output path

df.write.mode("overwrite").partitionBy("Region").format("csv").option("header", "true").save("abfss://staging@rawstorage123.dfs.core.windows.net/processed_sales_data.csv")

display(df)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3271535854854490>, line 17
      7 df = df.filter(col("Quantity") > 0)
      9 # Add Transaction Year
     10 #df = df.withColumn("TransactionYear", year(col("TransactionDate")))
     11 
   (...)
     14 
     15 #output_path = "abfss://staging@rawstorage123.dfs.core.windows.net/processed_sales_data.csv"  # Replace with your desired output path
---> 17 df.write.mode("overwrite").partitionBy("Region").format("csv").option("header", "true").save("abfss://staging@rawstorage123.dfs.core.windows.net/processed_sales_data.csv")
     19 display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, func